In [1]:
%load_ext autoreload

%autoreload 2

from helper import *

2023-03-31 23:07:52.896528: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-31 23:07:53.077644: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 23:07:53.748355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-03-31 23:08:16.049044: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [25]:
import csv
 

duplicate_list = []
original_list = []
question_list = []
list_idx_to_question_id = {}
question_id_to_list_idx = {}

# opening the CSV file
with open('dataset_subset.csv', mode ='r') as file:
   
  # reading the CSV file
  csvFile = csv.reader(file)
  
  index = 0
  # displaying the contents of the CSV file
  for i, row in enumerate(csvFile):
    if row == []:
        continue
    print(i)
    if i >= 1:
      # q1_title,q1_body,q1_id,q2_title,q2_body,q2_id,duplicate_label
      question_embedding = get_paragraph_embedding_bert(row[0])

      question_list.append(question_embedding)
      list_idx_to_question_id[index] = row[2]
      question_id_to_list_idx[row[2]] = index

      if row[6] == "1":
          duplicate_list.append([row[2], question_embedding])
          original_list.append(row[5])    
      index += 1  



0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
504
506
508
510
512
514
516
518
520
522
524
526

In [27]:
print(len(duplicate_list))
print(len(original_list))
print(len(question_list))

500
500
1000


In [51]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Function to return mean reciprocal rank from the model for the given list of question ids and associated list of duplicate "gold standard" question ids
# The score for an item in the question_ids list will be 1 if the first result given by the model matches the corresponding duplicate_ids item
# Searches k top results for each question
# Uses the basic BERT embedding from the helper.py file
# index_list is list of [id, embedding] pairs for every question in dataset
def get_mrr_and_accuracy(question_ids, duplicate_ids, k, index_list):
    mrr = 0.0
    accuracy = 0.0
    num_samples = len(question_ids)

    knn = NearestNeighbors(n_neighbors=10, metric='cosine')
    knn.fit(index_list)

    for i in range(num_samples):

        question_embedding = question_ids[i][1]
        duplicate_id = duplicate_ids[i]

        nearest_i = knn.kneighbors(question_embedding.reshape(1, -1), n_neighbors=k, return_distance=False).tolist()[0]

        # u = AnnoyIndex(768, 'angular')
        # u.load(annoy_test_set_filename)
        # nearest_i = u.get_nns_by_vector(question_embedding, k) # will return the index of the k nearest neighbor
        nearest_i = [list_idx_to_question_id[id] for id in nearest_i]
        print(duplicate_id)
        print(nearest_i)
        answer_rank = 0
        try:
            answer_rank = nearest_i.index(duplicate_id)
        except ValueError:
            # skip rest of iteration and don't update scores if the answer isn't found in the results
            continue
        
        print(answer_rank)
        mrr += 1.0 / answer_rank
        accuracy += 1 if answer_rank == 1.0 else 0
        
    mrr /= num_samples
    accuracy /= num_samples
    
    return mrr, accuracy
        

In [52]:
print(get_mrr_and_accuracy(duplicate_list, original_list, 5, question_list))

5490
['23', '23', '18556', '18556', '25138']
287721
['118', '118', '6735', '6735', '24680']
56
['331', '331', '17821', '17821', '29606']
5174
['665', '665', '18378', '18378', '6039']
170
['861', '861', '14537', '14537', '30189']
152
['1019', '1019', '20173', '20173', '20871']
866
['1029', '1029', '22723', '22723', '14115']
380700
['1472', '1472', '16983', '16983', '20168']
385
['1688', '1688', '25152', '25152', '17651']
96162
['2183', '2183', '35343', '35343', '32283']
113541
['2232', '2232', '17083', '17083', '8777']
1047
['2234', '2234', '20262', '20262', '9709']
531
['2423', '2423', '16984', '16984', '25212']
92
['2466', '2466', '13758', '13758', '11149']
46381
['2608', '2608', '7841', '7841', '36708']
660
['2705', '2705', '21406', '21406', '22235']
1016
['2736', '2736', '33788', '33788', '36945']
172
['2847', '2847', '26262', '26262', '14679']
8
['3004', '3004', '22072', '22072', '15099']
287721
['3178', '3178', '34331', '34331', '20773']
1095
['3215', '3215', '30336', '30336', '95

In [45]:
#feature choice will either be "title" or "body", but we can add a "both" option later
def get_all_question_embeddings1(feature_choice="body"):
    #embeddings will be array of shape [num_questions, 2], where each row will be: [question_id, question_embedding]
    embeddings = np.empty([len(questions), 2], dtype=object)

    print("STARTING")

    index = 0
    for id, value in questions.items():
        print(index)
        question_vec = get_paragraph_embedding_bert(value[feature_choice])
        #embeddings[index] = float(id)
        embeddings[index] = [id, question_vec]
        index += 1
    return embeddings

In [46]:
embeddings = get_all_question_embeddings1("body")

STARTING
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
2

KeyboardInterrupt: 

In [ ]:
print(len(embeddings))

In [ ]:
#use sklearn's nearest neighbor function to get the k nearest neighbors of given embedding vector
def get_k_nearest_neighbors(question_vector):

In [ ]:
# set a seed 
# t.set_seed(487)
# we need this to tell what the original sentences are 

# t.build(10) # build 10 trees, allegedly the more the better  
# t.save('bert_embedding_unnormalize.ann')

# Find k-nearest neighbor
k = 3 # first set k = 5
test_sentence = "Why did Old English lose both thorn and eth?" # one of the questions on linguistic stack exchange
test_sentence_vector = encode_sentence(test_sentence)

f = 768
u = AnnoyIndex(f, 'angular')
u.load('bert_embedding1.ann') 
nearest_i = u.get_nns_by_vector(test_sentence_vector, k) # will return the index of the k nearest neighbor
nearest_word = [(i, questions[i]) for i in nearest_i] # 
nearest_word